# Big Data Analysis Project - Group Project FS2020
## University of Zurich
## 5th of June 2020

Jara Fuhrer, 15-702-889

Claudio Brasser,

Severin Siffert, 14-720-536

Andrea Giambonini, 

Elizabeth Oladipo,


## "The goal of the project is to apply the data science pipeline"

![title](doingDS.png)

# Data Collection / Acquisition

Our dataset comes from http://www.kave.cc/, where 15'000 hours of interaction data from Visual Studio was collected in almost 11 million events from voluntaries.
To see what kind of data is collected, you can look at the data schema here: http://www.kave.cc/feedbag/event-generation.
Our goal is to analyze the link between the probability of Unit tests or project builds succeeding in connection with how much time has passed since the last build/test run.

The data was originally available in individual json events, with great support for parsing with Java or C#. Since we have to work with python, we used Java to convert the relevant information into csv without cleaning the data first.

# Data Exploration

<i> what to write here?? </i>
- chapter 5: EDA
- try to answer basic questions:
    - who constructed data set, when, why? trying to see relations btw working behaviour of developer (edit events, time, run test and results of them) and his efficienty / performance / productivity / habits /...
- what do the field mean? 
    - lets describe here quickly the data sets and their fields - will also help us for understanding
- basic statistics, min, max, median, quartile elements
- pairwise correlations: matrix of correlation coefficients
- create dot plots of different variables --> spot shape of distribution, outliers

### Final Data Analysis Questions:
1) linking time since last (successful?) build to probability of build succeeding -- SEVERIN

2) linking time since last tests run to probability of tests passing -- ELIZABETH

3) linking number of edit events since last successful build to probability of build succeeding -- ANDREA

4) linking number of edit events since last passing tests to probability of unit tests passing -- JARA


In [48]:
import pandas as pd
import numpy as np
import math
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from itertools import islice
from sklearn import datasets, linear_model
from sklearn.model_selection import KFold


In [2]:
df_edit = pd.read_csv("../data/df_edit.csv")
df_test = pd.read_csv("../data/df_test.csv")
df_build = pd.read_csv("../data/df_build.csv")

## TODO: Short description of the data frames & their columns

### Edit Events

In [3]:
df_edit.iloc[:,1:3]

,sessionID,timestamp
0,0,2016-10-04 14:36:01
1,0,2016-10-04 14:36:07
2,0,2016-10-04 14:36:49
3,0,2016-10-04 14:36:53
4,1,2016-10-04 14:37:03
...,...,...
497454,ffcbdaa4-e264-45a3-bddb-6f2f0afeac2f,2016-04-20 15:13:48
497455,ffcbdaa4-e264-45a3-bddb-6f2f0afeac2f,2016-04-20 15:13:52
497456,ffcbdaa4-e264-45a3-bddb-6f2f0afeac2f,2016-04-20 15:13:58
497457,ffcbdaa4-e264-45a3-bddb-6f2f0afeac2f,2016-04-20 15:14:03


In [4]:
df_edit["timestamp"].describe() 
 

count                  497459
unique                 488049
top       2016-09-04 23:28:12
freq                        4
Name: timestamp, dtype: object

In [5]:
df_edit["sessionID"].describe()

count                                   497459
unique                                    2876
top       8d0ea603-57cd-4b1f-b3cf-ce39ec9203c7
freq                                     17006
Name: sessionID, dtype: object

### Test Events

In [6]:
df_test.iloc[:,1:5]

,sessionID,timestamp,totalTests,testsPassed
0,006eb9aa-33f1-4e9e-8e74-7c978b58ee4a,2016-05-03 09:32:16,33,33
1,03c83bf2-8938-4a8f-9f58-d52bf3b2eccd,2016-05-10 17:21:18,1,0
2,03c83bf2-8938-4a8f-9f58-d52bf3b2eccd,2016-05-10 17:21:54,26,26
3,03c83bf2-8938-4a8f-9f58-d52bf3b2eccd,2016-05-10 17:28:26,1,1
4,0504fbd1-cce2-4431-b4e2-edc63eea1c6d,2016-07-13 20:24:46,21,21
...,...,...,...,...
3821,ffc444d0-8382-4e52-9f04-3c42601ec739,2016-06-14 14:57:23,1,0
3822,ffc444d0-8382-4e52-9f04-3c42601ec739,2016-06-14 15:04:16,1,0
3823,ffc444d0-8382-4e52-9f04-3c42601ec739,2016-06-14 15:05:03,1,0
3824,ffc444d0-8382-4e52-9f04-3c42601ec739,2016-06-14 15:23:52,1,1


In [7]:
df_test.describe()

,Unnamed: 0,totalTests,testsPassed
count,3826.000000,3826.000000,3826.000000
mean,1912.500000,88.780972,76.577627
std,1104.615393,430.785770,392.217908
min,0.000000,-1.000000,-1.000000
25%,956.250000,1.000000,0.000000
50%,1912.500000,3.000000,1.000000
75%,2868.750000,21.000000,15.000000
max,3825.000000,6618.000000,6090.000000


### Build Events

In [8]:
df_build.iloc[:,1:4]

,sessionID,timestamp,buildSuccessful
0,0,2016-10-04 14:35:55,False
1,0,2016-10-04 14:36:07,False
2,0,2016-10-04 14:36:50,False
3,0,2016-10-04 14:36:53,False
4,1,2016-10-04 14:37:03,True
...,...,...,...
14952,ffc444d0-8382-4e52-9f04-3c42601ec739,2016-06-14 15:04:17,True
14953,ffc444d0-8382-4e52-9f04-3c42601ec739,2016-06-14 15:05:05,True
14954,ffc444d0-8382-4e52-9f04-3c42601ec739,2016-06-14 15:23:53,True
14955,ffc444d0-8382-4e52-9f04-3c42601ec739,2016-06-14 15:24:13,True


In [9]:
df_build.describe()

,Unnamed: 0
count,14957.000000
mean,7478.000000
std,4317.858323
min,0.000000
25%,3739.000000
50%,7478.000000
75%,11217.000000
max,14956.000000


# Data Preprocessing / Cleaning

<i> --> TODO Claudio </i>

# Model / Algorithm Building

- chapter 6
- we should agree on some basics
- fit model with optimization methods?
- linear vs non-linear, blackbox vs descriptive, first principle vs data driven, stochastic vs deterministic, flat vs hierarchical
- model evaluation: trianign data set, validation data set, test data set
- classifiers, value prediction, absolute / relative / squared error, baseline models?

In [10]:
grouped = df_edit.groupby(['sessionID'])
sessionStarts = grouped.agg({'timestamp':np.min}).to_dict()

In [11]:
##Predicting Build success by time since last build

In [30]:
#build events are ordered by session id and timestamp
previous = {'sessionID': 'nonexistent-atsirtsakitaiea'}
time_to_build = []
for _, event in df_build.iterrows():
    try:
        if previous['sessionID'] is event['sessionID']:
            begin = previous['timestamp']
        else:
            begin = sessionStarts['timestamp'][event['sessionID']]
        end = event['timestamp']
        duration = pd.Timedelta(pd.to_datetime(end)-pd.to_datetime(begin)).seconds
        time_to_build.append((duration,event['buildSuccessful']))
    except Exception:
        #nothing
        ;
    previous = event
time_to_build

new session
 
duration 86394
first: {'sessionID': 'nonexistent-atsirtsakitaiea'}
second Unnamed: 0                       14061
sessionID                            0
timestamp          2016-10-04 14:35:55
buildSuccessful                  False
Name: 0, dtype: object
new session
new session
new session
 
duration 3331
first: Unnamed: 0                                        10278
sessionID          00479c7a-a779-4b46-b6e7-e421eabeddf3
timestamp                           2016-11-10 09:27:01
buildSuccessful                                    True
Name: 6, dtype: object
second Unnamed: 0                                        10279
sessionID          00479c7a-a779-4b46-b6e7-e421eabeddf3
timestamp                           2016-11-10 10:22:32
buildSuccessful                                    True
Name: 7, dtype: object
 
duration 2154
first: Unnamed: 0                                        10279
sessionID          00479c7a-a779-4b46-b6e7-e421eabeddf3
timestamp                           20

 
duration 2356
first: Unnamed: 0                                        12015
sessionID          02468b19-35f6-44a2-9d09-009a2b6d92c2
timestamp                           2016-06-29 12:20:46
buildSuccessful                                    True
Name: 186, dtype: object
second Unnamed: 0                                        12016
sessionID          02468b19-35f6-44a2-9d09-009a2b6d92c2
timestamp                           2016-06-29 13:00:02
buildSuccessful                                    True
Name: 187, dtype: object
 
duration 5331
first: Unnamed: 0                                        12026
sessionID          02468b19-35f6-44a2-9d09-009a2b6d92c2
timestamp                           2016-06-29 13:36:32
buildSuccessful                                    True
Name: 197, dtype: object
second Unnamed: 0                                        12027
sessionID          02468b19-35f6-44a2-9d09-009a2b6d92c2
timestamp                           2016-06-29 15:05:23
buildSuccessful          

 
duration 65447
first: Unnamed: 0                                         6486
sessionID          05294a7c-68d9-45e8-91c7-0ca5d154e4b5
timestamp                           2016-08-18 17:34:59
buildSuccessful                                    True
Name: 315, dtype: object
second Unnamed: 0                                         6487
sessionID          05294a7c-68d9-45e8-91c7-0ca5d154e4b5
timestamp                           2016-08-19 11:45:46
buildSuccessful                                    True
Name: 316, dtype: object
 
duration 3075
first: Unnamed: 0                                         6487
sessionID          05294a7c-68d9-45e8-91c7-0ca5d154e4b5
timestamp                           2016-08-19 11:45:46
buildSuccessful                                    True
Name: 316, dtype: object
second Unnamed: 0                                         6488
sessionID          05294a7c-68d9-45e8-91c7-0ca5d154e4b5
timestamp                           2016-08-19 12:37:01
buildSuccessful         

 
duration 1824
first: Unnamed: 0                                         6282
sessionID          06cc6270-977b-4a0c-add4-d409f1a8a128
timestamp                           2016-08-11 16:25:47
buildSuccessful                                    True
Name: 434, dtype: object
second Unnamed: 0                                         6283
sessionID          06cc6270-977b-4a0c-add4-d409f1a8a128
timestamp                           2016-08-11 16:56:11
buildSuccessful                                   False
Name: 435, dtype: object
 
duration 3140
first: Unnamed: 0                                         6284
sessionID          06cc6270-977b-4a0c-add4-d409f1a8a128
timestamp                           2016-08-11 16:57:11
buildSuccessful                                    True
Name: 436, dtype: object
second Unnamed: 0                                         6285
sessionID          06cc6270-977b-4a0c-add4-d409f1a8a128
timestamp                           2016-08-11 17:49:31
buildSuccessful          

 
duration 5949
first: Unnamed: 0                                         9225
sessionID          083c67aa-3e7d-4d78-8286-e28a85d6e1cc
timestamp                           2016-05-09 14:19:38
buildSuccessful                                    True
Name: 641, dtype: object
second Unnamed: 0                                         9226
sessionID          083c67aa-3e7d-4d78-8286-e28a85d6e1cc
timestamp                           2016-05-09 15:58:47
buildSuccessful                                    True
Name: 642, dtype: object
 
duration 1094
first: Unnamed: 0                                         9245
sessionID          083c67aa-3e7d-4d78-8286-e28a85d6e1cc
timestamp                           2016-05-09 16:21:09
buildSuccessful                                    True
Name: 661, dtype: object
second Unnamed: 0                                         9246
sessionID          083c67aa-3e7d-4d78-8286-e28a85d6e1cc
timestamp                           2016-05-09 16:39:23
buildSuccessful          

...

Name: 14308, dtype: object
 
duration 75599
first: Unnamed: 0                                         1174
sessionID          eff564bc-3daf-4a51-8a51-5e6e31aea6bf
timestamp                           2016-08-30 10:26:03
buildSuccessful                                    True
Name: 14313, dtype: object
second Unnamed: 0                                         1175
sessionID          eff564bc-3daf-4a51-8a51-5e6e31aea6bf
timestamp                           2016-08-31 07:26:02
buildSuccessful                                    True
Name: 14314, dtype: object
new session
new session
 
duration 6210
first: Unnamed: 0                                         4347
sessionID          f00c467e-8072-4099-a592-8e2dea7caf63
timestamp                           2016-08-04 15:42:11
buildSuccessful                                   False
Name: 14338, dtype: object
second Unnamed: 0                                         7153
sessionID          f054ccac-f2ce-48d0-8b14-6ad524ae3ae4
timestamp              

new session
new session
 
duration 74521
first: Unnamed: 0                                         1401
sessionID          f332e815-424e-4808-91ea-5fdf0b73b64c
timestamp                           2016-09-02 15:23:39
buildSuccessful                                    True
Name: 14453, dtype: object
second Unnamed: 0                                         1404
sessionID          f332e815-424e-4808-91ea-5fdf0b73b64c
timestamp                           2016-09-05 12:05:40
buildSuccessful                                    True
Name: 14454, dtype: object
 
duration 70027
first: Unnamed: 0                                         1412
sessionID          f332e815-424e-4808-91ea-5fdf0b73b64c
timestamp                           2016-09-05 12:13:44
buildSuccessful                                    True
Name: 14462, dtype: object
second Unnamed: 0                                         1414
sessionID          f332e815-424e-4808-91ea-5fdf0b73b64c
timestamp                           2016-09-06 07

 
duration 1857
first: Unnamed: 0                                         6533
sessionID          f89a3a50-efec-4411-a5d6-33199f14059d
timestamp                           2016-04-04 15:29:57
buildSuccessful                                    True
Name: 14646, dtype: object
second Unnamed: 0                                         6534
sessionID          f89a3a50-efec-4411-a5d6-33199f14059d
timestamp                           2016-04-04 16:00:54
buildSuccessful                                    True
Name: 14647, dtype: object
 
duration 2173
first: Unnamed: 0                                         6545
sessionID          f89a3a50-efec-4411-a5d6-33199f14059d
timestamp                           2016-04-04 16:30:11
buildSuccessful                                    True
Name: 14658, dtype: object
second Unnamed: 0                                         6546
sessionID          f89a3a50-efec-4411-a5d6-33199f14059d
timestamp                           2016-04-04 17:06:24
buildSuccessful    

first: Unnamed: 0                                         4105
sessionID          fb8003b5-9a89-4371-81b6-779eef8a15b2
timestamp                           2016-12-08 23:24:28
buildSuccessful                                    True
Name: 14821, dtype: object
second Unnamed: 0                                         4112
sessionID          fb8003b5-9a89-4371-81b6-779eef8a15b2
timestamp                           2016-12-09 02:19:05
buildSuccessful                                    True
Name: 14822, dtype: object
 
duration 2571
first: Unnamed: 0                                         4112
sessionID          fb8003b5-9a89-4371-81b6-779eef8a15b2
timestamp                           2016-12-09 02:19:05
buildSuccessful                                    True
Name: 14822, dtype: object
second Unnamed: 0                                         4113
sessionID          fb8003b5-9a89-4371-81b6-779eef8a15b2
timestamp                           2016-12-09 03:01:56
buildSuccessful                    

[(86394, False),
 (12, False),
 (43, False),
 (3, False),
 (0, True),
 (146, False),
 (520, True),
 (3331, True),
 (2154, True),
 (572, True),
 (1795, True),
 (216, True),
 (50, True),
 (1154, True),
 (1683, True),
 (1493, True),
 (188, True),
 (451, True),
 (204, True),
 (114, True),
 (310, True),
 (146, True),
 (51, True),
 (265, False),
 (137, False),
 (116, False),
 (119, False),
 (52, True),
 (176, True),
 (94, True),
 (42, True),
 (194, True),
 (170, True),
 (38, True),
 (90, False),
 (37, True),
 (90, True),
 (196, True),
 (93, True),
 (82, True),
 (138, True),
 (405, True),
 (70, True),
 (412, True),
 (289, True),
 (26, True),
 (2711, True),
 (544, False),
 (50, False),
 (100, True),
 (19, True),
 (114, False),
 (24, False),
 (27, False),
 (57, False),
 (61, False),
 (8, False),
 (8, False),
 (74, False),
 (301, True),
 (81287, True),
 (9707, False),
 (20, True),
 (30, True),
 (79399, True),
 (42, True),
 (15, True),
 (10, True),
 (35, True),
 (11, True),
 (4056, True),
 (524, 

In [38]:
frame = pd.DataFrame(time_to_build, columns=['time','success'])
X_train, X_test, y_train, y_test = train_test_split(time_to_build, frame['success'], test_size=0.2)
print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

11963 11963
2991 2991


In [47]:
lm = linear_model.LinearRegression()
model = lm.fit(X_train, y_train)
predictions = lm.predict(X_test)
                        
print('Score:', model.score(X_test, y_test))

Score: 1.0


# Data Visualization

- we should agree on some basic design / plots such that we have a consistent viz